In [22]:
%cd C:\Users\abiga\OneDrive\Documents\GitHub\PyTorch-Object-Detection-Faster-RCNN-Tutorial
!cd

C:\Users\abiga\OneDrive\Documents\GitHub\PyTorch-Object-Detection-Faster-RCNN-Tutorial
C:\Users\abiga\OneDrive\Documents\GitHub\PyTorch-Object-Detection-Faster-RCNN-Tutorial


In [24]:
%run training_script.py

AssertionError: No files found in path: C:\Users\abiga\OneDrive\Documents\GitHub\PyTorch-Object-Detection-Faster-RCNN-Tutorial\src\pytorch_faster_rcnn_tutorial\data\heads\input

In [2]:
# imports
import ast
import os
import pathlib

import neptune
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from pytorch_faster_rcnn_tutorial.backbone_resnet import ResNetBackbones
from pytorch_faster_rcnn_tutorial.datasets import (
    ObjectDetectionDataSet,
    ObjectDetectionDatasetSingle,
)
from pytorch_faster_rcnn_tutorial.faster_RCNN import get_faster_rcnn_resnet
from pytorch_faster_rcnn_tutorial.transformations import (
    ComposeDouble,
    ComposeSingle,
    FunctionWrapperDouble,
    FunctionWrapperSingle,
    apply_nms,
    apply_score_threshold,
    normalize_01,
)
from pytorch_faster_rcnn_tutorial.utils import (
    collate_single,
    get_filenames_of_path,
    save_json,
)
from pytorch_faster_rcnn_tutorial.viewers.object_detection_viewer import (
    ObjectDetectionViewer,
    ObjectDetectionViewerSingle,
)
from training_script import NeptuneSettings

In [3]:
# parameters
params = {
    "EXPERIMENT": "GM-Thesis3",  # experiment name, e.g. Head-42
    "OWNER": "mikmikthemimic",  # e.g. johndoe55
    "INPUT_DIR": "pytorch_faster_rcnn_tutorial/data/heads/test",  # files to predict
    "PREDICTIONS_PATH": "predictions",  # where to save the predictions
    "MODEL_DIR": "heads",  # load model from checkpoint
    "DOWNLOAD": True,  # whether to download from neptune
    "DOWNLOAD_PATH": "model",  # where to save the model if DOWNLOAD is True
    "PROJECT": "GM-Thesis3",  # Project name
}

In [4]:
# input files
inputs = get_filenames_of_path(pathlib.Path(params["INPUT_DIR"]))
inputs.sort()

2024-04-20 11:54:28 - INFO - utils.py:27:get_filenames_of_path - Found 8 files in pytorch_faster_rcnn_tutorial\data\heads\test


In [5]:
# transformations
transforms = ComposeSingle(
    [
        FunctionWrapperSingle(np.moveaxis, source=-1, destination=0),
        FunctionWrapperSingle(normalize_01),
    ]
)

In [6]:
# create dataset
dataset = ObjectDetectionDatasetSingle(
    inputs=inputs,
    transform=transforms,
    use_cache=False,
)

In [7]:
# create dataloader
dataloader_prediction = DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_single,
)

In [8]:
# environment variables (pydantic BaseSettings class)
neptune_settings: NeptuneSettings = NeptuneSettings()

In [20]:
# import experiment from neptune
project_name = f'{params["OWNER"]}/{params["PROJECT"]}'
project = neptune.init(
    project_qualified_name=project_name, api_token=neptune_settings.api_key
)  # get project
experiment_id = params["EXPERIMENT"]  # experiment id
experiment = neptune.init_model(with_id=experiment_id)
parameters = experiment["parameters"].fetch()
properties = experiment.get_properties()

AttributeError: module 'neptune' has no attribute 'init_model'

In [ ]:
# rcnn transform
transform = GeneralizedRCNNTransform(
    min_size=int(parameters["MIN_SIZE"]),
    max_size=int(parameters["MAX_SIZE"]),
    image_mean=ast.literal_eval(parameters["IMG_MEAN"]),
    image_std=ast.literal_eval(parameters["IMG_STD"]),
)

In [ ]:
# color mapping
color_mapping = {
    1: "red",
}

In [ ]:
# view dataset
datasetviewer = ObjectDetectionViewerSingle(
    dataset=dataset, color_mapping=color_mapping, rcnn_transform=transform
)

In [ ]:
# download model from neptune or load from checkpoint
if params["DOWNLOAD"]:
    download_path = pathlib.Path(os.getcwd()) / params["DOWNLOAD_PATH"]
    download_path.mkdir(parents=True, exist_ok=True)
    model_name = "best_model.pt"  # that's how I called the best model
    # model_name = properties['checkpoint_name']  # logged when called log_model_neptune()
    if not (download_path / model_name).is_file():
        experiment.download_artifact(
            path=model_name, destination_dir=download_path
        )  # download model

    model_state_dict = torch.load(
        download_path / model_name, map_location=torch.device("cpu")
    )
else:
    checkpoint = torch.load(params["MODEL_DIR"], map_location=torch.device("cpu"))
    model_state_dict = checkpoint["hyper_parameters"]["model"].state_dict()

In [ ]:
model = get_faster_rcnn_resnet(
    num_classes=int(parameters["CLASSES"]),
    backbone_name=ResNetBackbones(parameters["BACKBONE"]),  # reverse look-up enum
    anchor_size=ast.literal_eval(parameters["ANCHOR_SIZE"]),
    aspect_ratios=ast.literal_eval(parameters["ASPECT_RATIOS"]),
    fpn=ast.literal_eval(parameters["FPN"]),
    min_size=int(parameters["MIN_SIZE"]),
    max_size=int(parameters["MAX_SIZE"]),
)

In [ ]:
# load weights
model.load_state_dict(model_state_dict)

In [ ]:
# inference (cpu)
model.eval()
for sample in dataloader_prediction:
    x, x_name = sample
    with torch.no_grad():
        pred = model(x)
        pred = {key: value.numpy() for key, value in pred[0].items()}
        name = pathlib.Path(x_name[0])
        save_dir = pathlib.Path(os.getcwd()) / params["PREDICTIONS_PATH"]
        save_dir.mkdir(parents=True, exist_ok=True)
        pred_list = {
            key: value.tolist() for key, value in pred.items()
        }  # numpy arrays are not serializable -> .tolist()
        save_json(pred_list, path=save_dir / name.with_suffix(".json"))

In [ ]:
# get prediction files
predictions = get_filenames_of_path(
    pathlib.Path(os.getcwd()) / params["PREDICTIONS_PATH"]
)
predictions.sort()

In [ ]:
# create prediction dataset
iou_threshold = 0.25
score_threshold = 0.6

transforms_prediction = ComposeDouble(
    [
        FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
        FunctionWrapperDouble(normalize_01),
        FunctionWrapperDouble(
            apply_nms, input=False, target=True, iou_threshold=iou_threshold
        ),
        FunctionWrapperDouble(
            apply_score_threshold,
            input=False,
            target=True,
            score_threshold=score_threshold,
        ),
    ]
)

dataset_prediction = ObjectDetectionDataSet(
    inputs=inputs, targets=predictions, transform=transforms_prediction, use_cache=False
)


In [ ]:
# mapping
color_mapping = {
    1: "red",
}

In [ ]:
# visualize predictions
datasetviewer_prediction = ObjectDetectionViewer(
    dataset=dataset_prediction, color_mapping=color_mapping
)